In [1]:
import pandas as pd
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord

In [74]:
def resultados_lncRNA(resultFinal, nameI):
    lncRNA = resultFinal[resultFinal['queryName']=="lncRNA"]
    lncRNA = lncRNA.reset_index(drop=True)
    if lncRNA.empty == False:
        lncRNA.to_csv(nameI+'_lncRNAs.csv', index=False)
        gfflncRNA = salvar_gff(lncRNA, nameI+'_lncRNA')
        QtdTotallncRNA = lncRNA['ncRNA'].value_counts(dropna=False, sort=True)
        QtdTotallncRNA = QtdTotallncRNA.to_frame().reset_index()
        QtdTotallncRNA = QtdTotallncRNA.rename(columns= {'ncRNA': 'Qtd', 'index':'ncRNA'})
        
        lncRNA2 = lncRNA[['ncRNA', 'IDdb']]
        lncRNA2 = lncRNA2.drop_duplicates()
        lncRNA2 = lncRNA2.set_index('ncRNA')
        lncRNAdict = lncRNA2.to_dict()
        QtdTotallncRNA['IDdb'] = QtdTotallncRNA['ncRNA'].map(lncRNAdict['IDdb'])
        
        QtdTotallncRNA.to_csv(nameI+'_Tabela_quantidade_lncRNAs.csv')
        if salvar_Fasta != None:
            salvar_fasta(gfflncRNA, nameI+'_lncRNA', list(lncRNA['ncRNA']))
        

In [46]:
###Salva em GFF        
def salvar_gff(resultFinalFinal, nameI):      
    if os.path.exists(nameI+'_ResultadoFinal.gff'):
        os.remove(nameI+'_ResultadoFinal.gff')
    
    gff = pd.DataFrame(columns=['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes'])
    gff['seq_id'] = resultFinalFinal['targetName']
    gff['source'] = resultFinalFinal['source']
    gff['type'] = resultFinalFinal['queryName']
    gff['start'] = resultFinalFinal['seqFrom']
    gff['end'] = resultFinalFinal['seqTo']
    gff['score'] = resultFinalFinal['score']
    gff['strand'] = resultFinalFinal['strand']
    gff['phase'] = '.'
    gff['attributes'] = resultFinalFinal['ID'] #resultado do blast/infernal
    gff['attributes']=gff['attributes'].str.replace('id', 'ID=id')
    gff.to_csv(nameI+'_ResultadoFinal.gff', sep='\t',mode='a', index=False, header=False)
    
    file = open(nameI+'_ResultadoFinal.gff', 'r')
    lines = file.readlines()
    file.close()
    lines.insert(0, "##gff-version 3"+'\n')
    
    file = open(nameI+"_ResultadoFinal.gff", 'w')
    file.writelines(lines)
    file.close()
    
    return gff

In [81]:
##Salvar sequencia dos ncRNAs em FASTA
def salvar_fasta(gff, nameI, miRNA):
    ids = list(gff['seq_id'])
    start = list(gff['start'])
    end = list(gff['end'])
    attr = list(gff['attributes'].str.replace('ID=id',''))
    attrOri = list(gff['attributes'])
    tipo = list(gff['type'])
    i=1
    frags=[]
    for record in SeqIO.parse('Beatriz/NCBI/Bmusculus/GCF_009873245.2_mBalMus1.pri.v3_genomic.fna', "fasta"):
        for i in range(len(ids)):
            if record.id == ids[i]:
                frag = record.seq[start[i]-1:end[i]]
                gravar = SeqRecord(frag, "fragment_" + str(attr[i]), ids[i], tipo[i]+';'+attrOri[i]+';'+miRNA[i])
                frags.append(gravar)
    SeqIO.write(frags, nameI+"_ncRNA.fa", "fasta")

In [2]:
arq = pd.read_csv('Beatriz/NCBI/Bmusculus/BmusculusFinal_Resultado_Final.csv', skiprows=1, names=['targetName', 'seqFrom', 'seqTo', 'pident', 'qcover', 'IDdb', 'strand', 'score', 'Evalue','ncRNA','queryName', 'source', 'ID'])
#arq['targetName'] = arq['targetName'].map(dici)
arq

,targetName,seqFrom,seqTo,pident,qcover,IDdb,strand,score,Evalue,ncRNA,queryName,source,ID
0,Seq_1,66769,66840,CL00001,-,RF00005,-,36.8,9.000000e-01,tRNA,tRNA,Infernal,id1
1,Seq_1,94504,94577,CL00001,-,RF00005,-,35.1,2.700000e+00,tRNA,tRNA,Infernal,id2
2,Seq_1,97105,97177,CL00001,-,RF00005,+,39.5,1.700000e-01,tRNA,tRNA,Infernal,id3
3,Seq_1,103374,103444,CL00001,-,RF00005,+,39.5,1.600000e-01,tRNA,tRNA,Infernal,id4
4,Seq_1,193161,193233,CL00001,-,RF00005,-,46.4,1.900000e-03,tRNA,tRNA,Infernal,id5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29499,Seq_9,107362057,107362163,CL00009,-,RF00026,+,77.4,1.200000e-13,U6,splicing,Infernal,id29500
29500,Seq_9,107415824,107415896,CL00001,-,RF00005,-,54.1,7.900000e-06,tRNA,tRNA,Infernal,id29501
29501,Seq_9,107416039,107416113,CL00001,-,RF00005,-,31.9,1.300000e+01,tRNA,tRNA,Infernal,id29502
29502,Seq_92,47187,47259,CL00001,-,RF00005,+,37.0,2.300000e-04,tRNA,tRNA,Infernal,id29503


In [84]:
salvar_Fasta=1
resultados_lncRNA(arq,'Beatriz/NCBI/Bmusculus/BmusculusFinal')


In [72]:
def resultados_miRNA(resultFinal, nameI):
    miRNA = resultFinal[(resultFinal['queryName']=="miRNA") | (resultFinal['queryName']=="pre-miRNA")]
    miRNA = miRNA.reset_index(drop=True)
    if miRNA.empty == False:
        miRNA.to_csv(nameI+'_miRNAs.csv', index=False)
        gffmiRNA = salvar_gff(miRNA, nameI+'_miRNA')
        QtdTotalmiRNA = miRNA['ncRNA'].value_counts(dropna=False, sort=True)
        QtdTotalmiRNA = QtdTotalmiRNA.to_frame().reset_index()
        QtdTotalmiRNA = QtdTotalmiRNA.rename(columns= {'ncRNA': 'Qtd', 'index':'ncRNA'})
        
        miRNA2 = miRNA[['ncRNA', 'IDdb']]
        miRNA2 = miRNA2.drop_duplicates()
        miRNA2 = miRNA2.set_index('ncRNA')
        miRNAdict = miRNA2.to_dict()
        QtdTotalmiRNA['IDdb'] = QtdTotalmiRNA['ncRNA'].map(miRNAdict['IDdb'])
        
        QtdTotalmiRNA.to_csv(nameI+'_Tabela_quantidade_miRNAs.csv')
        if salvar_Fasta != None:    
            salvar_fasta(gffmiRNA, nameI+'_miRNA', list(miRNA['ncRNA']))

In [ ]:
resultados_miRNA(arq,'Beatriz/NCBI/Bmusculus/BmusculusFinal')

In [82]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import argparse
import pandas as pd
import pickle

argsFasta = 'Beatriz/NCBI/Bmusculus/GCF_009873245.2_mBalMus1.pri.v3_genomic.fna'
outargsFasta = 'Beatriz/NCBI/Bmusculus/New_GCF_009873245.2_mBalMus1.pri.v3_genomic.fna'

frags=[]
ids=[]
n=1
for record in SeqIO.parse(argsFasta, 'fasta'):
    ids.append(record.id)
    frag = record.seq
    gravar = SeqRecord(frag, 'Seq_'+str(n), "", "")
    frags.append(gravar)
    n=n+1
SeqIO.write(frags, outargsFasta, 'fasta')

seqDmel=[]
for i in range(len(ids)):
    seqDmel.append('Seq_'+str(i+1))
dfDmel = pd.DataFrame(columns=['Seq','Original'])
dfDmel.Original=ids
dfDmel.Seq = seqDmel

dici=dict(dfDmel.values)

with open("Beatriz/NCBI/Bmusculus/FASTA_Temp.pkl", "wb") as tf:
    pickle.dump(dici,tf)